In [1]:
from PIL import Image
import numpy as np                         
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
import json
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/Machine learning/data.zip"
!unzip "/content/drive/MyDrive/Machine learning/data.zip"

Archive:  /content/drive/MyDrive/Machine learning/data.zip
   creating: data/
   creating: data/cortical_lesion/
   creating: data/cortical_lesion/P001/
  inflating: data/cortical_lesion/P001/P001.10006.bmp  
  inflating: data/cortical_lesion/P001/P001.10007.bmp  
  inflating: data/cortical_lesion/P001/P001.10008.bmp  
  inflating: data/cortical_lesion/P001/P001.10009.bmp  
  inflating: data/cortical_lesion/P001/P001.10010.bmp  
  inflating: data/cortical_lesion/P001/P001.10011.bmp  
  inflating: data/cortical_lesion/P001/P001.10012.bmp  
  inflating: data/cortical_lesion/P001/P001.10013.bmp  
  inflating: data/cortical_lesion/P001/P001.10014.bmp  
  inflating: data/cortical_lesion/P001/P001.10015.bmp  
  inflating: data/cortical_lesion/P001/P001.10016.bmp  
  inflating: data/cortical_lesion/P001/P001.10017.bmp  
  inflating: data/cortical_lesion/P001/P001.10018.bmp  
  inflating: data/cortical_lesion/P001/P001.10019.bmp  
  inflating: data/cortical_lesion/P001/P001.10020.bmp  
  infla

In [4]:
colours = ( (255, 255, 255, "(255, 255, 255)"),
            (255, 0, 0, "(255, 0, 0)"),
            (0, 255, 0, "(0, 255, 0)"))

def nearest_colour( subjects, query ):
    return min( subjects, key = lambda subject: sum( (s - q) ** 2 for s, q in zip( subject, query ) ) )

def create_sub_masks(mask_image):
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]
            nearest_color = nearest_colour(colours,pixel)[3]
            # If the pixel is not white...(we have white background)
            if nearest_color != "(255, 255, 255)":
                # Check to see if we've created a sub-mask...
                
                pixel_str = nearest_color
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks

In [5]:
def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    #     measure.find_contours
    #Uses the “marching squares” method to compute a the iso-valued contours of 
    #the input 2D array for a particular level value. 
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')
    bboxes = []
    segmentations = []
    polygons = []
    areas = []
    annotations = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        #shapely.geometry Polygon
        
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        if len(segmentation)>1:
          segmentations.append([segmentation])
          polygons.append(poly)
    # Combine the polygons to calculate the bounding box and area
    offset = 0
    steps = 0
    for i , polygon in enumerate(polygons):
        bbox = []
        area = []
        if not polygon.is_empty:
            bbox = polygon.bounds
            area = polygon.area
        annotation = {
        'segmentation': segmentations[i],
        'iscrowd': 0,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area}
        annotations.append(annotation)
        steps +=1
        annotation_id += 1
    return annotations
        
        

In [6]:
errors=[
 ##To many colors masks...
 './data/cortical_lesion/P064/P064.10009.bmp', 
 './data/cortical_lesion/P018/P018.10016.bmp',
 #weird white mask
 './data/cortical_lesion/P003/P003.10011.bmp', #(255,250,250)
 #many weird masks
 './data/cortical_lesion/P003/P003.10013.bmp',
 #Multipolygo problem
 './data/cortical_lesion/P001/P001.10008.bmp',
 './data/cortical_lesion/P001/P001.10010.bmp',#(255, 97, 97)
 './data/cortical_lesion/P001/P001.10009.bmp'#(255, 164, 164)
 
 ]
errors=[]

In [7]:
coco_annotations={
    'info':{
        'description':'Dataset of brain lesions',
        'url':"",
        'year':2020,
        "contributor":"Dr. Martin Tabakow",
        "date_created": ""
    }, 
    'licenses':{
        'url':"",
        "id":0,
        'name':""
    },
    'images':[],
    'annotations':[],
    'categories':[]
}

In [8]:
coco_annotations['categories'].append({
    'supercategory':'brain damage',
    'id':1,
    'name':'cortical lesion'
})
coco_annotations['categories'].append({
    'supercategory':'brain damage',
    'id':2,
    'name':'subcortical lesion'
})

In [9]:
category_ids = {
        '(255, 0, 0)': 1,
        '(0, 255, 0)': 2
}

In [10]:
import os
from tqdm import tqdm
data_path = "./data/"
out_path = "/content/drive/MyDrive/Machine learning/brain_annotations.json"

with open(out_path,'w') as json_file:            
    json.dump(coco_annotations,json_file)

lesion_id = 1
is_crowd= 0
image_id = 1
annotation_id = 1
for (root,dirs,files) in tqdm(os.walk(data_path, topdown=True)):
    for filename in files:
        if filename.endswith('.bmp'):
          path = os.path.join(root, filename)
          if path not in errors:
            image={
            'license':0,
            'file_name':filename,
            'width':512,
            'height':512,
            'id':image_id
            }
            with Image.open(path) as mask_image:
              sub_masks = create_sub_masks(mask_image)
              for color, sub_mask in sub_masks.items():
               
                try:
                  category_id = category_ids[color]
                
                  annotations = create_sub_mask_annotation(sub_mask, image_id, category_id,annotation_id, is_crowd)
                except Exception as e:
                  print(e)
                  print(color,path)
                  continue
                annotation_id += len(annotations)
                for annotation in annotations:
                  with open(out_path,'r') as outfile:
                    data=json.load(outfile)
                    data['annotations'].append(annotation)
                  with open(out_path, "w") as outfile:
                    json.dump(data,outfile)
              with open(out_path,'r') as outfile:
                    data=json.load(outfile)
                    data['images'].append(image)
              with open(out_path, "w") as outfile:
                    json.dump(data,outfile)
              image_id+=1
           


69it [21:29, 128.48s/it]

'MultiPolygon' object has no attribute 'exterior'
(255, 0, 0) ./data/cortical_lesion/P032/P032.10013.bmp


70it [25:22, 159.92s/it]

'MultiPolygon' object has no attribute 'exterior'
(255, 0, 0) ./data/cortical_lesion/P006/P006.10011.bmp


76it [47:30, 257.78s/it]

...

118it [4:12:46, 821.98s/it]

'MultiPolygon' object has no attribute 'exterior'
(255, 0, 0) ./data/cortical_lesion/P016/P016.10018.bmp
'MultiPolygon' object has no attribute 'exterior'
(255, 0, 0) ./data/cortical_lesion/P016/P016.10012.bmp


121it [4:25:31, 651.91s/it]

'MultiPolygon' object has no attribute 'exterior'
(255, 0, 0) ./data/cortical_lesion/P058/P058.10006.bmp
'MultiPolygon' object has no attribute 'exterior'
(255, 0, 0) ./data/cortical_lesion/P058/P058.10016.bmp


125it [4:55:24, 141.79s/it]
